In [ ]:
# !pip install datasets transformers
!pip install transformers[torch]
!pip install datasets transformers==4.28.0
!pip install accelerate -U
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgra

In [ ]:
import transformers
import os
import json
from transformers import AutoTokenizer
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
path = "/content/abcd_v1.1.json"

file = open(path)
dictionary = json.load(file)

print(dictionary.keys())

dict_keys(['train', 'dev', 'test'])


In [ ]:
# compiling all convos into a list
convos = []
for i in range(0,len(dictionary['train'])):
    convo = ""
    for sub in dictionary['train'][i]['original']:
        if sub[0] == "customer":
            convo += f"Customer: {sub[1]} "
        if sub[0] == "agent":
            convo += f"Agent: {sub[1]} "
    convos.append(convo)

In [ ]:
dev_convos = []
for i in range(0,len(dictionary['dev'])):
    convo = ""
    for sub in dictionary['dev'][i]['original']:
        if sub[0] == "customer":
            convo += f"Customer: {sub[1]} "
        if sub[0] == "agent":
            convo += f"Agent: {sub[1]} "
    dev_convos.append(convo)

In [ ]:
with open("output.txt", "w") as txt_file:
      txt_file.write('\n'.join(convos)) # works with any number of elements in a line

In [ ]:
with open("validate.txt", "w") as txt_file:
      txt_file.write('\n'.join(dev_convos)) # works with any number of elements in a line

In [ ]:
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}
datasets = load_dataset("mlerner27/asapp2a", data_files={"train": "output.txt", "validation": "validate.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/8034 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1004 [00:00<?, ? examples/s]

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 1024

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/8034 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1004 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

" kind of expensive Agent: Yes it is expensive. Customer: okay, well that's good to know Agent: But they will come fast. Agent: Do you need anymore help? Customer: maybe I will just wear some old clothes to the party Customer: no, that's it Agent: Okay have a great day! Customer: i just needed to know about the shipping Agent: hello! How can I help you today? Customer: I want an extension for my shopping service. Customer: I need more time to get the money. Agent: So you need to postpone payment for your subscription? Customer: yes Agent: Can I have your full name and account ID please Customer: Chloe Zhang Customer: F5KCD5QPAW Agent: Ok. I pulled up your account. Now I need your order ID please Customer: 6820094438 Agent: Are you a Gold or Silver member? Customer: silver Agent: I can only extend your payment by one day. Customer: Okay Agent: Ok. Your extension is now reflected on your account. Agent: Is there anything else I can help you with today? Customer: no Agent: I hope you have

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    per_device_train_batch_size=1,
    eval_accumulation_steps=1,
    do_eval=True
)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]', })
    # tokenizer.add_special_tokens(["[Agent: ]"])
    # tokenizer.add_tokens('Agent: ', special_tokens=True)
    model.resize_token_embeddings(len(tokenizer))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    tokenizer=tokenizer,
)

Using pad_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/EfazAhmed/distilgpt2-finetuned-2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.6k/319M [00:00<?, ?B/s]

Download file runs/Oct11_01-23-02_2bd2d560c858/events.out.tfevents.1696987395.2bd2d560c858.713.0: 100%|#######…

Download file runs/Nov13_20-26-53_d674103e4323/events.out.tfevents.1699907293.d674103e4323.968.0: 100%|#######…

Download file training_args.bin: 100%|##########| 3.93k/3.93k [00:00<?, ?B/s]

...

Clean file training_args.bin:  25%|##5       | 1.00k/3.93k [00:00<?, ?B/s]

Clean file runs/Oct11_01-23-02_2bd2d560c858/1696987395.51264/events.out.tfevents.1696987395.2bd2d560c858.713.1…

Download file runs/Nov13_20-26-53_d674103e4323/1699907293.708259/events.out.tfevents.1699907293.d674103e4323.9…

Clean file runs/Nov13_20-26-53_d674103e4323/1699907293.708259/events.out.tfevents.1699907293.d674103e4323.968.…

Clean file pytorch_model.bin:   0%|          | 1.00k/319M [00:00<?, ?B/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.320000,2.166151
2,2.127100,2.067333
3,2.090300,2.042823


TrainOutput(global_step=5283, training_loss=2.289363239381477, metrics={'train_runtime': 1655.1798, 'train_samples_per_second': 3.192, 'train_steps_per_second': 3.192, 'total_flos': 1380430733377536.0, 'train_loss': 2.289363239381477, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Perplexity: 1532048903295141823905792.00
